# Telecom Churn Case Study

In the telecom industry, customers are able to choose from multiple service providers and actively switch from one operator to another. In this highly competitive market, the telecommunications industry experiences an average of 15-25% annual churn rate. Given the fact that it costs 5-10 times more to acquire a new customer than to retain an existing one, customer retention has now become even more important than customer acquisition.

For many incumbent operators, retaining high profitable customers is the number one business goal.
To reduce customer churn, telecom companies need to predict which customers are at high risk of churn.

### Business Objective
The business objective is to predict the churn in the last (i.e. the ninth) month using the data (features) from the first three months. Also recommend strategies to manage customer churn based on your observations

## Import Libraries

In [ ]:
# pip install fancyimpute
import pandas as pd
import numpy as np
from fancyimpute import IterativeImputer
from sklearn import preprocessing
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import RFE
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

# Read the Data

In [ ]:
telecom_data = pd.read_csv('telecom_churn_data.csv')
print('No of rows within the telecom data is -',len(telecom_data))

## View the Percentage of Missing values withina column

In [ ]:
((telecom_data.isnull().sum()/len(telecom_data))*100).sort_values(ascending=False)

## Filter High Valued Customer
- In the below defined steps we are calculating the high value customers by using both data usage and recharge amount
- So we calculate total recharge for data by multiplying total recharge date and average recharge amount of data
- Then we calculate average of both the 6th and 7th month defining 'good' phase
- Those peoples who have their average more than 70th percentile value of average of 6 and 7th month are named as <b>High Valued Customer<b>

In [ ]:
## Derive Total Data Recharge Amt
telecom_data["total_rech_data_amt_6"] = telecom_data["total_rech_data_6"]  * telecom_data['av_rech_amt_data_6']
telecom_data["total_rech_data_amt_7"] = telecom_data["total_rech_data_7"]  * telecom_data['av_rech_amt_data_7']
telecom_data["total_rech_data_amt_8"] = telecom_data["total_rech_data_8"]  * telecom_data['av_rech_amt_data_8']
telecom_data["total_rech_data_amt_9"] = telecom_data["total_rech_data_9"]  * telecom_data['av_rech_amt_data_9']

## Drop total_rech_data_* and av_rech_amt_data_*
drop_col = ["total_rech_data_6", "total_rech_data_7", "total_rech_data_8", "total_rech_data_9", 
                'av_rech_amt_data_6', 'av_rech_amt_data_7', 'av_rech_amt_data_8', 'av_rech_amt_data_9']
telecom_data.drop(drop_col, axis=1, inplace=True)

av_rech_amt_6n7 = (telecom_data["total_rech_amt_6"].fillna(0) + telecom_data["total_rech_data_amt_6"].fillna(0) + 
telecom_data["total_rech_amt_7"].fillna(0) + telecom_data["total_rech_data_amt_7"].fillna(0))/2.0

percentile_70 = np.percentile(av_rech_amt_6n7,70)
print('The 70 Percentile value for the month of 6 and 7 is -',percentile_70)

telecom_data = telecom_data[(av_rech_amt_6n7>=percentile_70)]
print('The No. of rows after filtering the data based on 70th Percentile -',len(telecom_data))

## Data Cleaning

### Remove Fatures with Single Unique Value

- Features with sigle unique values does not impact on churn
- So it doesn't make sense to keep them

In [ ]:
## Remove Data which has only 1 unique Value
telecom_data1 = telecom_data.loc[:,telecom_data.apply(pd.Series.nunique) != 1]

### Convert Column Names to Numeric to keep it Standardised Throughout

In [ ]:
telecom_data1 = telecom_data1.rename(columns={'aug_vbc_3g':'vbc_3g_8','jul_vbc_3g':'vbc_3g_7','jun_vbc_3g':'vbc_3g_6','sep_vbc_3g':'vbc_3g_9'})

### Check for Object Columns

In [ ]:
# Convert the data type of columns in telecom_data1 to Data Time
object_data = telecom_data1.select_dtypes(include='object')
for col in object_data.columns:
    telecom_data1[col] = pd.to_datetime(telecom_data1[col])

### Drop Columns with > 30 % Missing Values

In [ ]:
percentage_missing = pd.DataFrame({'Percent_Missing':telecom_data1.isnull().sum()/len(telecom_data1)})
cols_drop = percentage_missing[(percentage_missing['Percent_Missing']>0.3)&(~percentage_missing.index.str.contains('_9'))].index
telecom_data2 = telecom_data1.drop(cols_drop,axis=1)
telecom_data2.dropna(how='all',axis=0).shape

In [ ]:
# Find Columns with Unique Value but Insignificant Frequency
# If one the the frequency of one of the values for a column is more than 95%, drop that column
for col_name in telecom_data2.columns:
    if (len(telecom_data2[col_name].unique()) <= 8):
        print(telecom_data2[col_name].value_counts())
        print(f"\n{35 * '-'}")

### Drop Highly Correlated Variables

In [ ]:
## Find the correlated variables 
correlation = telecom_data2.corr()
upper = correlation.where(np.triu(np.ones(correlation.shape),k=1).astype(bool))
# Drop columns with more than 80 percent correlation with other variables
to_drop = [column for column in upper.columns if any(upper[column]>0.80)] 

In [ ]:
# Drop columns with high correlations, ['mobile_number'] as well other columns of not significance
to_drop = ['mobile_number','loc_og_t2m_mou_6','std_og_t2t_mou_6','std_og_t2t_mou_7','std_og_t2t_mou_8','std_og_t2t_mou_9','std_og_t2m_mou_6',
                'std_og_t2m_mou_7','std_og_t2m_mou_8','std_og_t2m_mou_9','total_og_mou_6','total_og_mou_7','total_og_mou_8',
                'loc_ic_t2t_mou_6','loc_ic_t2t_mou_7','loc_ic_t2t_mou_8','loc_ic_t2t_mou_9','loc_ic_t2m_mou_6','loc_ic_t2m_mou_7','loc_ic_t2m_mou_8','loc_ic_t2m_mou_9',
                'std_ic_t2m_mou_6','std_ic_t2m_mou_7','std_ic_t2m_mou_8','std_ic_t2m_mou_9','total_ic_mou_6','total_ic_mou_7','total_ic_mou_8',
                'total_rech_amt_6','total_rech_amt_7','total_rech_amt_8','total_rech_amt_9','arpu_2g_9','count_rech_2g_9','count_rech_3g_9','vol_3g_mb_6','vol_3g_mb_7','vol_3g_mb_8',
                'loc_og_t2t_mou_6','loc_og_t2t_mou_7','loc_og_t2t_mou_8','loc_og_t2t_mou_9','loc_og_t2f_mou_6','loc_og_t2f_mou_7','loc_og_t2f_mou_8','loc_og_t2f_mou_9',
                'loc_og_t2m_mou_6','loc_og_t2m_mou_7','loc_og_t2m_mou_8','loc_og_t2m_mou_9','loc_ic_t2f_mou_6','loc_ic_t2f_mou_7','loc_ic_t2f_mou_8','loc_ic_t2f_mou_9'
               ,'date_of_last_rech_6','arpu_6','arpu_7','arpu_9','date_of_last_rech_7','date_of_last_rech_8','date_of_last_rech_9','date_of_last_rech_data_9','night_pck_user_9','arpu_3g_9','max_rech_data_9','total_rech_data_amt_9','fb_user_9']

telecom_data3 = telecom_data2.drop(to_drop,axis=1)

telecom_data3.shape

In [ ]:
#telecom_data3.dtypes

## Tag Churn
- The churned customers are the ones who hav not used any voice or data in the 9th month
- If a customer is churned, the customer will be tagged as 1. Else 0
- We have don't a mistake of not considering data consumption for 9th month and also reversly tagged the churning ****

In [ ]:
#telecom_data3['churn'] = np.where(telecom_data3[['vol_2g_mb_9','vol_3g_mb_9','count_rech_2g_9', 'count_rech_3g_9']].sum(axis=1)==0,0,1)
telecom_data3['churn'] = np.where(telecom_data3[['vol_2g_mb_9','vol_3g_mb_9']].sum(axis=1)==0,0,1)
#Remove All 9th Month related columns
drop_cols = [col for col in telecom_data3.columns if '_9' in col]
print(drop_cols)

telecom_data3.drop(drop_cols, axis=1, inplace=True)

telecom_data3.shape

## Missing Value Treatment

In [ ]:
# # Remove date columns (starts with: date)
# telecom_data3.drop([col for col in telecom_data3.columns if 'date_' in col], axis=1, inplace=True)
# telecom_data3 = telecom_data3.drop(telecom_data3.columns[telecom_data3.dtypes=='datetime64[ns]'],axis=1)
telecom_data3 = telecom_data3.drop(telecom_data3.columns[telecom_data3.dtypes=='<M8[ns]'],axis=1)

### Impute Missing Values using IterativeImputer from fancyimpute

In [ ]:
telecom_data3_columns = telecom_data3.columns
ii = IterativeImputer()
df_clean = pd.DataFrame(ii.fit_transform(telecom_data3))
df_clean.columns = telecom_data3_columns

In [ ]:
df_clean.head()

In [ ]:
# Distribution graphs (histogram/bar graph) of column data
def plotPerColumnDistribution(df, nGraphShown, nGraphPerRow):
    nunique = df.nunique()
    df = df[[col for col in df if nunique[col] > 1 and nunique[col] < 50]] # For displaying purposes, pick columns that have between 1 and 50 unique values
    nRow, nCol = df.shape
    columnNames = list(df)
    nGraphRow = (nCol + nGraphPerRow - 1) / nGraphPerRow
    plt.figure(num = None, figsize = (6 * nGraphPerRow, 8 * nGraphRow), dpi = 80, facecolor = 'w', edgecolor = 'k')
    for i in range(min(nCol, nGraphShown)):
        plt.subplot(nGraphRow, nGraphPerRow, i + 1)
        columnDf = df.iloc[:, i]
        if (not np.issubdtype(type(columnDf.iloc[0]), np.number)):
            valueCounts = columnDf.value_counts()
            valueCounts.plot.bar()
        else:
            columnDf.hist()
        plt.ylabel('counts')
        plt.xticks(rotation = 90)
        plt.title(f'{columnNames[i]} (column {i})')
    plt.tight_layout(pad = 1.0, w_pad = 1.0, h_pad = 1.0)
    plt.show()
  
# Correlation matrix
def plotCorrelationMatrix(df, graphWidth):
    filename = "Telecom Churn"
    df = df.dropna('columns') # drop columns with NaN
    df = df[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    if df.shape[1] < 2:
        print(f'No correlation plots shown: The number of non-NaN or constant columns ({df.shape[1]}) is less than 2')
        return
    corr = df.corr()
    plt.figure(num=None, figsize=(graphWidth, graphWidth), dpi=80, facecolor='w', edgecolor='k')
    corrMat = plt.matshow(corr, fignum = 1)
    plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
    plt.yticks(range(len(corr.columns)), corr.columns)
    plt.gca().xaxis.tick_bottom()
    plt.colorbar(corrMat)
    plt.title(f'Correlation Matrix for {filename}', fontsize=15)
    plt.show()
# Scatter and density plots
def plotScatterMatrix(df, plotSize, textSize):
    df = df.select_dtypes(include =[np.number]) # keep only numerical columns
    # Remove rows and columns that would lead to df being singular
    df = df.dropna('columns')
    df = df[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    columnNames = list(df)
    if len(columnNames) > 10: # reduce the number of columns for matrix inversion of kernel density plots
        columnNames = columnNames[:10]
    df = df[columnNames]
    ax = pd.plotting.scatter_matrix(df, alpha=0.75, figsize=[plotSize, plotSize], diagonal='kde')
    corrs = df.corr().values
    for i, j in zip(*plt.np.triu_indices_from(ax, k = 1)):
        ax[i, j].annotate('Corr. coef = %.3f' % corrs[i, j], (0.8, 0.2), xycoords='axes fraction', ha='center', va='center', size=textSize)
    plt.suptitle('Scatter and Density Plot')
    plt.show()


## Distribution graphs (histogram/bar graph) of sampled columns:

In [ ]:
plotPerColumnDistribution(df_clean, 10, 5)

## Correlation Matrix

In [ ]:
plotCorrelationMatrix(df_clean, 53)

## Scatter and Density plot

In [ ]:
plotScatterMatrix(df_clean, 20, 10)

## Reduce the No of Columns by Creating New Meaningful Features
Create Columns with Average of 6th & 7th Month Since it's a "Good" Phase and Keep the 8th month untouched as it's "Action" Phase, for now to see if it can give any additional insight

In [ ]:
col_list = df_clean.filter(regex='_6|_7').columns.str[:-2]
col_list.unique()

for idx, col in enumerate(col_list.unique()):
    print(col)
    avg_col_name = "avg_"+col+"_av67"
    col_6 = col+"_6"
    col_7 = col+"_7"
    df_clean[avg_col_name] = (df_clean[col_6]  + df_clean[col_7])/ 2



# print (df_high_val_cust.shape)

In [ ]:
print (df_clean.shape)

col_list = df_clean.filter(regex='_6|_7').columns

df_clean.drop(col_list, axis=1, inplace=True)
df_clean.shape

In [ ]:
#Conevrt AON in Months
df_clean['aon_mon'] = df_clean['aon']/30
df_clean.drop('aon', axis=1, inplace=True)
df_clean['aon_mon'].head()

## Churn Distribution

In [ ]:
#Churn Distribution
ax = (df_clean['churn'].value_counts()*100.0 /len(df_clean)).plot.pie(autopct='%.1f%%', labels = ['No', 'Yes'],figsize =(5,5), fontsize = 12 )                                                                           

ax.set_ylabel('Churn',fontsize = 12)
ax.set_title('Churn Distribution', fontsize = 12)

In our data, 54% of the customers do not churn. So, the data is slighly imbalanced

## Distribution graphs (histogram/bar graph) of sampled columns:

In [ ]:
plotPerColumnDistribution(df_clean, 10, 5)

## Correlation Matrix

In [ ]:
plotCorrelationMatrix(df_clean, 53)

## Scatter and density plot

In [ ]:
plotScatterMatrix(df_clean, 20, 10)

In [ ]:
import seaborn as sns
ax = sns.distplot(df_clean['aon_mon'], hist=True, kde=False, 
             bins=int(180/5), color = 'darkblue', 
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 4})
ax.set_ylabel('No of Customers')
ax.set_xlabel('Tenure (months)')
ax.set_title('Customers by their tenure')

In [ ]:
tn_range = [0, 6, 12, 24, 60, 61]
tn_label = [ '0-6 Months', '6-12 Months', '1-2 Yrs', '2-5 Yrs', '5 Yrs and above']
df_clean['tenure_range'] = pd.cut(df_clean['aon_mon'], tn_range, labels=tn_label)
df_clean['tenure_range'].head()

In [ ]:
sns.set()
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})

temp = pd.Series(data = 'tenure_range')
fig, ax = plt.subplots()
width = len(df_clean['tenure_range'].unique()) + 6 + 4*len(temp.unique())
fig.set_size_inches(width , 7)

total = float(len(df_clean.index))
ax = sns.countplot(x="tenure_range", data=df_clean, palette="Set2", hue = "churn");
for p in ax.patches:
                ax.annotate('{:1.1f}%'.format((p.get_height()*100)/float(len(df_clean))), (p.get_x()+0.05, p.get_height()+20))
plt.xticks(rotation=90)
plt.show()

## Correlation of "Churn" with other variables:


In [ ]:
#Get Correlation of "Churn" with other variables:
plt.figure(figsize=(20,10))
df_clean.corr()['churn'].sort_values(ascending = False).plot(kind='bar')

Avg STD Outgoing Calls for Month 6 & 7, Outgoing calls in Roaming seems to be positively correlated with Churn while Avg Revenue, No Of Recharge for 8th Month seems negatively correlated.



Lets look at the relation between total recharge in 8th Month Vs Average Revenue in 8th Month

In [ ]:
df_clean[['total_rech_num_8', 'arpu_8']].plot.scatter(x = 'total_rech_num_8',
                                                              y='arpu_8')

Lets look at the relation between Tenure And Revenue

## Churn vs Tenure 

In [ ]:
sns.boxplot(x = df_clean.churn, y = df_clean.aon_mon)


As we can see form the below plot, the customers who do not churn, they tend to stay for a longer tenure with the telecom company.

## Churn Vs Volume based cost

In [ ]:
ax = sns.kdeplot(df_clean.avg_max_rech_amt_av67[(df_clean["churn"] == 0)],
                color="Red", shade = True)
ax = sns.kdeplot(df_clean.avg_max_rech_amt_av67[(df_clean["churn"] == 1)],
                ax =ax, color="Blue", shade= True)
ax.legend(["Not Churn","Churn"])
ax.set_ylabel('Density')
ax.set_xlabel('Volume based cost')
ax.set_title('Distribution of Volume based cost by churn')

## Churn Vs Max Recharge Amount

In [ ]:
ax = sns.kdeplot(df_clean.max_rech_amt_8[(df_clean["churn"] == 0)],
                color="Red", shade = True)
ax = sns.kdeplot(df_clean.max_rech_amt_8[(df_clean["churn"] == 1)],
                ax =ax, color="Blue", shade= True)
ax.legend(["Not Churn","Churn"])
ax.set_ylabel('Density')
ax.set_xlabel('Volume based cost')
ax.set_title('Distribution of Max Recharge Amount by churn')

* People Who Recharge with less Amount are more likely to Churn
* There is no visible difference in Volume Based Cost & Churn

In [ ]:
#Lets Create New DF for Model Building

df = df_clean[:].copy()

#Dropping tenure_range since we have AON MONTH already and columns are highly coorelated
df.drop('tenure_range', axis=1, inplace=True)

#Since All The Values are realted to Price/ Cost/ Amount, Filling NaN with 0

df.fillna(0, inplace=True)

df.head()

In [ ]:
X = df.drop(['churn'], axis=1)
y = df['churn']

df.drop('churn', axis=1, inplace=True)

## Feature Scaling
 - Use StandardScaler

In [ ]:

scaler = preprocessing.StandardScaler().fit(X)
X = scaler.transform(X)

## Train Test Split

In [ ]:
# Split in train & Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, train_size=0.7, random_state=1)

In [ ]:
print("X_train Shape : ", X_train.shape)
print("X_test Shape : ", X_test.shape)

y_train_imb = (y_train != 0).sum()/(y_train == 0).sum()
y_test_imb = (y_test != 0).sum()/(y_test == 0).sum()
print("Imbalance in Train Data : ", y_train_imb)
print("Imbalance in Test Data : ", y_test_imb)

### Use SMOTE for Balancing Dataset

In [ ]:
# Balancing DataSet
sm = SMOTE(kind = "regular")
X_tr,y_tr = sm.fit_sample(X_train,y_train)

In [ ]:
print("X_tr Shape", X_tr.shape)
print("y_tr Shape", y_tr.shape)

imb = (y_tr != 0).sum()/(y_tr == 0).sum()
print("Imbalance in Train Data : ",imb)

## Modelling

### SVM

In [ ]:
lr = LogisticRegression()
lr.svm = SVC(kernel = 'linear')
lr.svm.fit(X_tr,y_tr)
pred = lr.svm.predict(X_test)
accuracy_score(y_test,pred)

## RFE

In [ ]:
# Reduce the features to 20 using Recursive Feature Eliminination
rfe = RFE(lr,20)
rfe = rfe.fit(X_tr,y_tr)
df.columns[rfe.support_]

In [ ]:
  X_rfe = pd.DataFrame(X_tr).iloc[:,rfe.support_]
  y_rfe = y_tr
  lr.fit(X_rfe,y_rfe)
  X_test_rfe = pd.DataFrame(X_test).iloc[:,rfe.support_]
  y_pred = lr.predict(X_test_rfe)

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
print('Accuracy of Logistic Regression Model on test set is ',accuracy_score(y_test, y_pred))

### Logistic Regression Model Summary

- Model Accuracy is 82.5%
- Confusion matrics clearly shows that the model has drawback in predicting high false positives.

## Principal Compound Analysis

In [ ]:
# Dimention Reduction using PCA
pca = PCA(random_state=100)
pca.fit(X_tr)

X_tr_pca = pca.transform(X_tr)
print(X_tr_pca.shape)

X_test_pca = pca.transform(X_test)
print(X_test_pca.shape)

## Logistic Regression

In [ ]:
lr_pca = LogisticRegression()
lr_pca.fit(X_tr_pca,y_tr)
y_pred = lr_pca.predict(X_test_pca)
confusion_matrix(y_test,y_pred)

In [ ]:
print('The Accuracy of Logistic Regression with PCA: ',accuracy_score(y_test,y_pred))

In [ ]:
#Making the screeplot - plotting the cumulative variance against the number of components
%matplotlib inline
fig = plt.figure(figsize = (12,8))
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')
plt.show()

In [ ]:
np.cumsum(np.round(pca.explained_variance_ratio_,decimals=4)*100)

In [ ]:
pca_32 = PCA(n_components=32)

df_tr_pca_32 = pca_32.fit_transform(X_tr)
print(df_tr_pca_32.shape)

df_test_pca_32 = pca_32.transform(X_test)
print(df_test_pca_32.shape)

In [ ]:
lr_pca1 = LogisticRegression(C=1e9)
lr_pca1.fit(df_tr_pca_32, y_tr)

# Predicted probabilities
y_pred32 = lr_pca1.predict(df_test_pca_32)

# Converting y_pred to a dataframe which is an array
df_y_pred = pd.DataFrame(y_pred32)

# Print Confusion Matrix
print(confusion_matrix(y_test,y_pred32))

# Print Accuracy for Logistic Regression with PCA
print('Logistic Regression accuracy with PCA ',accuracy_score(y_test,y_pred32))

## Decision Tree

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=1)

In [ ]:
sm = SMOTE(kind='regular')
X_tr,y_tr = sm.fit_sample(X_train,y_train)
print(X_tr.shape)
print(y_tr.shape)

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
 
lsvc = LinearSVC(C=0.001, penalty="l1", dual=False).fit(X_tr, y_tr)
model = SelectFromModel(lsvc, prefit=True)
X_lasso = model.transform(X_tr)
pos = model.get_support(indices=True)
 ### Feature reduction using RFE
print(X_lasso.shape)
print(pos)
#feature vector for decision tree
lasso_features = list(df.columns[pos])
print("Features for LASSO model buidling: ", lasso_features)

### Decision Tree With Default Parameters

In [ ]:
dt1 = DecisionTreeClassifier(max_depth=5)
dt1.fit(X_lasso,y_tr)
# Making predictions
X_test = pd.DataFrame(data=X_test).iloc[:, pos]
y_pred1 = dt1.predict(X_test)
print(classification_report(y_test,y_pred1))

In [ ]:
# Printing confusion matrix and accuracy
print(confusion_matrix(y_test,y_pred1))
print('Accuracy of Decision Tree :',accuracy_score(y_test,y_pred1))

### Hyper Parameter Tuning

In [ ]:
# GridSearchCV to find optimal max_depth
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV


# specify number of folds for k-fold CV
n_folds = 5

# parameters to build the model on
parameters = {'max_depth': range(1, 40)}

# instantiate the model
dtree = DecisionTreeClassifier(criterion = "gini", 
                               random_state = 100)

# fit tree on training data
tree = GridSearchCV(dtree, parameters, 
                    cv=n_folds, 
                   scoring="accuracy",return_train_score='warn')
tree.fit(X_train, y_train)

In [ ]:
# scores of GridSearch CV
score = tree.cv_results_
pd.DataFrame(score).head()

In [ ]:
# plotting accuracies with max_depth
plt.figure()
plt.plot(score["param_max_depth"], 
         score["mean_train_score"], 
         label="training accuracy")
plt.plot(score["param_max_depth"], 
         score["mean_test_score"], 
         label="test accuracy")
plt.xlabel("max_depth")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

According to the above plot, we can see the distribution of test and train accuracies for each value of max_depth within the range of 1 to 40.

# Tuning Min Samples Leaf

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

# specify number of folds for k-fold CV
n_folds = 5

# parameter to build the model on
parameters = {'min_samples_leaf':range(5,200,20)}

# instantiate the model
dtree = DecisionTreeClassifier(criterion='gini',random_state=100)

# fit tree on training data
tree = GridSearchCV(dtree,parameters,cv=n_folds,scoring='accuracy',return_train_score='warn')

tree.fit(X_lasso,y_tr)

score = tree.cv_results_
pd.DataFrame(score).head()

In [ ]:
plt.figure()
plt.plot(score['param_min_samples_leaf'],score['mean_train_score'],label='training accuracy')
plt.plot(score['param_min_samples_leaf'],score['mean_test_score'],label='test accuracy')
plt.xlabel("min sample leaf")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

min_samples_leaf = 25 looks to be optimal

# Tuning min samples split

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

nfolds = 5
parameters = {'min_samples_split':range(5,200,20)}
dtree = DecisionTreeClassifier(criterion='gini',random_state=100)
tree = GridSearchCV(dtree,parameters,cv=nfolds,scoring='accuracy',return_train_score='warn')
tree.fit(X_lasso,y_tr)

score = tree.cv_results_

plt.figure()
plt.plot(score['param_min_samples_split'],score['mean_train_score'],label='training accuracy')
plt.plot(score['param_min_samples_split'],score['mean_test_score'],label='test accuracy')
plt.legend()
plt.show()

# Grid Search for Optimal Values

In [ ]:
# Create the parameter grid 
param_grid = {
    'max_depth': range(5, 15, 5),
    'min_samples_leaf': range(25, 175, 50),
    'min_samples_split': range(50, 150, 50),
    'criterion': ["entropy", "gini"]
}

n_folds = 5

# Instantiate the grid search model
dtree = DecisionTreeClassifier()
grid_search = GridSearchCV(estimator = dtree, param_grid = param_grid, 
                          cv = n_folds, verbose = 1)

# Fit the grid search to the data
grid_search.fit(X_lasso, y_tr)
# cv results
cv_results = pd.DataFrame(grid_search.cv_results_)
cv_results
# printing the optimal accuracy score and hyperparameters
print("Best Accuracy", grid_search.best_score_)

In [ ]:
print(grid_search.best_estimator_)

In [ ]:
# model with optimal hyperparameters
clf_gini = DecisionTreeClassifier(criterion = "gini", 
                                  random_state = 100,
                                  max_depth=5, 
                                  min_samples_leaf=25,
                                  min_samples_split=50)
clf_gini.fit(X_lasso, y_tr)

In [ ]:
# accuracy score
print ('Accuracy Score for Decision Tree Final Model :',clf_gini.score(X_test,y_test))

In [ ]:
# plotting tree 
import pydotplus, graphviz
from IPython.display import Image  
from sklearn.externals.six import StringIO  
from sklearn.tree import export_graphviz
import pydotplus, graphviz

features = X_test.columns
dot_data = StringIO()  
export_graphviz(clf_gini, out_file=dot_data,feature_names=features,filled=True,rounded=True)

graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())

#### Summary - Decision Tress
* Getting around 84% accuracy 
* Confusion matix shows lot of false positives still exist.
* 31 Features were selected for Model Building

# Random Forest

Random Forest with default parameters

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

model_rf = RandomForestClassifier()
model_rf.fit(X_lasso,y_tr)

# Make Predictions
prediction_test = model_rf.predict(X_test)
print('Random Forest Accuracy with default hyperparameters',metrics.accuracy_score(y_test,prediction_test))

In [ ]:
## Printing the classification report viewing the accuracies
print(classification_report(y_test,prediction_test))

In [ ]:
## Printing the Confusion Matrix
print(confusion_matrix(y_test,prediction_test))  

### Hyper Parameter Tunning

In [ ]:
# GridSearchCV to find the optimal n_estimators
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

# specify no. of folds for k-fold cv
n_folds = 5

# parameter to build the model on
parameters = {'max_depth':range(2,20,5)}

rf = RandomForestClassifier()

# fit tree on training data
rf = GridSearchCV(rf,parameters,scoring='accuracy',return_train_score='warn')
rf.fit(X_lasso,y_tr)

score = rf.cv_results_

# Show the plot to identify optimal value
plt.figure()
plt.plot(score['param_max_depth'],score['mean_train_score'],label='traning accuracy')
plt.plot(score['param_max_depth'],score['mean_test_score'],label='test accuracy')
plt.xlabel("max_depth")
plt.ylabel('accuracy_score')
plt.legend()
plt.show()

## Tuning Min Sample Leaf

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

# specify number of folds for k fold cv
n_folds = 5

# parameters to build the model on
parameter = {'min_samples_leaf':range(50,400,10)}

# instantiate the model
rf = RandomForestClassifier()

# fit tree on training data
rf = GridSearchCV(rf,parameter,cv = n_folds,scoring='accuracy',return_train_score='warn')
rf.fit(X_lasso,y_tr)

# Scores of Grid Search CV
scores = rf.cv_results_

# Plotting accuracies with min sample leaf
plt.figure()
plt.plot(scores['param_min_samples_leaf'],scores['mean_train_score'],label='train accuracy')
plt.plot(scores['param_min_samples_leaf'],scores['mean_test_score'],label='test accuracy')
plt.xlabel('min_samples_leaf')
plt.ylabel('accuracy score')
plt.legend()
plt.show()

## Tuning Min Samples Split

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

# specify number of folds for k fold cv
n_folds = 5

# parameters to build the model on
parameter = {'min_samples_split':range(100,500,25)}

# instantiate the model
rf = RandomForestClassifier()

# fit tree on training data
rf = GridSearchCV(rf,parameter,cv = n_folds,scoring='accuracy',return_train_score='warn')
rf.fit(X_lasso,y_tr)

# Scores of Grid Search CV
scores = rf.cv_results_

# Plotting accuracies with min sample leaf
plt.figure()
plt.plot(scores['param_min_samples_split'],scores['mean_train_score'],label='train accuracy')
plt.plot(scores['param_min_samples_split'],scores['mean_test_score'],label='test accuracy')
plt.xlabel('min_samples_split')
plt.ylabel('accuracy score')
plt.legend()
plt.show()

## Grid Search to Find Optimal Hyperparameter

In [ ]:
# Create the parameter grid based on the results of random search
param_grid = {'max_depth':[4,8,10],'min_samples_leaf':range(100,300,100),'min_samples_split':range(200,500,100),'n_estimators':[500,700],'max_features':[10,20,25]}

# Create a base model
rf = RandomForestClassifier()

# Instantiate the grid search model
grid_search = GridSearchCV(estimator=rf,param_grid=param_grid,cv=3,)

In [ ]:
#Commenting as it takes long time
# Fit the grid search to the data
grid_search.fit(X_lasso, y_tr)
# printing the optimal accuracy score and hyperparameters
print('Accuracy is',grid_search.best_score_,'using',grid_search.best_params_)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)

model_rf = RandomForestClassifier(bootstrap=True,
                                  max_depth=10,
                                  min_samples_leaf=100, 
                                  min_samples_split=200,
                                  n_estimators=1000 ,
                                  oob_score = True, n_jobs = -1,
                                  random_state =50,
                                  max_features = 15,
                                  max_leaf_nodes = 30)
model_rf.fit(X_train, y_train)

# Make predictions
prediction_test = model_rf.predict(X_test)

In [ ]:
# evaluation metrics
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,prediction_test))
print(confusion_matrix(y_test,prediction_test))

In [ ]:

# accuracy score
print ('Accuracy Score for Random Forest Final Model :',metrics.accuracy_score(y_test, prediction_test))

In [ ]:
X = df
# Scaling all the variables to a range of 0 to 1
#from sklearn.preprocessing import MinMaxScaler
features = X.columns.values
X = pd.DataFrame(scaler.transform(X))
X.columns = features

importances = model_rf.feature_importances_
weights = pd.Series(importances,
                 index=X.columns.values)
weights.sort_values()[-10:].plot(kind = 'barh')

#### Observations
- The results from random forest are very similar to that of the logistic regression 
- From random forest algorithm, Local Incoming for Month 8, Average Revenue Per Customer for Month 8 and Max Recharge Amount for Month 8 are the most important predictor variables to predict churn

###  Final Recomendation : Telecom Churn
* Std Outgoing Calls and Revenue Per Customer are strong indicators of Churn
* People with less than 4 Yrs of Tenure are more likely to Churn
* Behaviour of Volume Based Cost is not a strong indicator of Churn
* Max Recharge Amount could be a good Churn Indicator
* Random Forest is the best method to Predict Churn followed by SVM, other models too do a fair job
* Behaviour in 8th Month can be the base of Churn Analysis
* Local Incoming and Outgoing Calls for 8th Month and Average Revenue in 8th Month are strong indicators of Churn Behaviour